### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [2]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function parabola(x)
    a = 0.13625841750841752
    b = 0.189709595959596
    c = 0.22
    return  a * (x.^2) + b * x .+ c 
end

function line(x)
    a = -0.2558139534883721
    b = -0.16046511627906984
    return a * x .+ b
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [3]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 3000.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [4]:
startI = -1.8
endI = -1.32
lengthI = 100
rangeI = range( startI, endI, length = 100 )
step_to_left = rangeI.step.hi

0.004848484848484658

In [5]:
file = "2d_map_betweeen_line_parabola.txt"

"2d_map_betweeen_line_parabola.txt"

In [ ]:
for (index_I0, I0) in enumerate(rangeI)
    
        if index_I0 == 1
            global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
        end
        
        U0 = parabola(I0)
    
        print_output(index_I0, I0, U0, initial_condition_function)
    
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    
        ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_function = tr[end, :]
    
        println("last point: $initial_condition_function");flush(stdout)
    
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 4, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
    
        println("----------------------")
        if I0  >  -1.8
            move_to_left( I0, U0, initial_condition_function, step_to_left )
        end
        
end

index: 1
I0: -1.8
U0: 0.32000000000000006
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [1.7303735312129922, 0.8546534767458698, 0.6819520373729314, 0.09103793623701394]
LSE: [0.00045279931611982827, -0.4584742933804628, -0.5556136735610765, -8.704974692499281]
----------------------
index: 2
I0: -1.7951515151515152
U0: 0.31854467849601
initial condition: [1.7303735312129922, 0.8546534767458698, 0.6819520373729314, 0.09103793623701394]
last point: [1.0486697820068387, 0.6124287315138137, 0.7774131582109282, 0.08304585378161278]
LSE: [0.00036172332160014446, -0.5167413091914771, -0.5553448575277006, -8.692111743781584]
----------------------

<<<<<<<<<<<<<<<<<<<<<
 <- 
index: 1
I0: -1.7999999999999998
U0: 0.31854467849601
initial condition: [1.0486697820068387, 0.6124287315138137, 0.7774131582109282, 0.08304585378161278]
last point: [0.9150143815319376, 0.9247912913711375, 0.5925230111024512, 0.4894138343536877]
LSE: [-0.3347300117843138, -0.33452609000071026, -12.242345234710735,

LSE: [-0.3393454316370679, -0.33941728329278514, -11.499744435350506, -15.226200311748459]
_________________

index: 10
I0: -1.7563636363636363
U0: 0.3071327322814926
initial condition: [4.729426700245686, 0.8118160521696745, 0.6719330727203612, 0.08296837239025434]
last point: [1.6855746548824242, 0.847616012264384, 0.6829099468487094, 0.07601245774830705]
LSE: [0.0008969511630960218, -0.5555556169050602, -0.7532408208626702, -8.403140861715995]
----------------------

<<<<<<<<<<<<<<<<<<<<<
 <- 
index: 1
I0: -1.761212121212121
U0: 0.3071327322814926
initial condition: [1.6855746548824242, 0.847616012264384, 0.6829099468487094, 0.07601245774830705]
last point: [18.211835976739618, 0.3697050258318577, 0.7792799606869044, 0.089161699891089]
LSE: [-9.860233437610947e-6, -0.3608250576207319, -0.555561892311306, -8.246977870869372]
_________________

 <- 
index: 2
I0: -1.7660606060606057
U0: 0.3071327322814926
initial condition: [18.211835976739618, 0.3697050258318577, 0.7792799606869044, 0

U0: 0.3043438084549195
initial condition: [1.3071821683195044, 0.832499749929778, 0.6989635785679981, 0.06939571395452974]
last point: [0.9876886317857041, 0.7780887359434412, 0.7239122060175213, 0.07543496573532714]
LSE: [-0.0002601952585317798, -0.45431856577887164, -0.5556968371336113, -8.199569447679913]
_________________

 <- 
index: 2
I0: -1.756363636363636
U0: 0.3043438084549195
initial condition: [0.9876886317857041, 0.7780887359434412, 0.7239122060175213, 0.07543496573532714]
last point: [0.9358626084281463, 0.9247108566359453, 0.57999339048885, 0.48911575910385463]
LSE: [-0.3203870544289007, -0.3204639067430698, -11.67549883739239, -14.973734294891983]
_________________

 <- 
index: 3
I0: -1.7612121212121206
U0: 0.3043438084549195
initial condition: [0.9358626084281463, 0.9247108566359453, 0.57999339048885, 0.48911575910385463]
last point: [0.9325116913840764, 0.9248271557887544, 0.5811053372729061, 0.48954670327794836]
LSE: [-0.32325815011676484, -0.32186796917969773, -11.62

LSE: [-0.33991842220000223, -0.34028881628341906, -11.281752227306335, -15.638092515243068]
_________________

 <- 
index: 12
I0: -1.7999999999999976
U0: 0.30295895595069144
initial condition: [0.908473556519429, 0.9257298509876757, 0.5887439773571022, 0.4928832589135586]
last point: [0.9052050098978178, 0.9258449631270791, 0.5898806892059931, 0.4933076595328564]
LSE: [-0.3428436716492949, -0.3427023953083783, -11.245691462119426, -15.701299858749664]
_________________

index: 14
I0: -1.736969696969697
U0: 0.30158050971916905
initial condition: [5.088715972464196, 0.8065202944484806, 0.6704113226205826, 0.07828910927305104]
last point: [1.363484362785489, 0.8345514682769307, 0.6947810264238387, 0.06807328663749761]
LSE: [0.0004392937049678336, -0.5555556137597341, -0.8069634995589443, -8.203980116845267]
----------------------

<<<<<<<<<<<<<<<<<<<<<
 <- 
index: 1
I0: -1.7418181818181817
U0: 0.30158050971916905
initial condition: [1.363484362785489, 0.8345514682769307, 0.694781026423838

U0: 0.3002084697603522
initial condition: [0.9298211322294382, 0.9251114111104604, 0.5804047233524264, 0.4905989775469159]
last point: [0.9265013455948508, 0.9252260666691369, 0.5815205320444666, 0.4910230013435364]
LSE: [-0.32666540806057837, -0.32666016752792393, -11.38036048503696, -15.48959466555952]
_________________

 <- 
index: 8
I0: -1.7709090909090894
U0: 0.3002084697603522
initial condition: [0.9265013455948508, 0.9252260666691369, 0.5815205320444666, 0.4910230013435364]
last point: [0.923191040351652, 0.9253406258930585, 0.5826394339632844, 0.49144642903414787]
LSE: [-0.32890093120727415, -0.32919746632571556, -11.34428055702206, -15.551797292444796]
_________________

 <- 
index: 9
I0: -1.775757575757574
U0: 0.3002084697603522
initial condition: [0.923191040351652, 0.9253406258930585, 0.5826394339632844, 0.49144642903414787]
last point: [0.9198899963797635, 0.9254551111438885, 0.5837613776782233, 0.49186934289917805]
LSE: [-0.33129892186751003, -0.3316937875918983, -11.3087

...

initial condition: [0.9312451350147314, 0.9254346858973331, 0.5768140130583098, 0.49179390873829193]
last point: [0.9279438930218264, 0.9255469883339486, 0.5779241207215675, 0.49220856709256305]
LSE: [-0.32643562171306345, -0.3259244956048392, -11.121523688220005, -16.020447199278973]
_________________

 <- 
index: 12
I0: -1.761212121212119
U0: 0.29211076173426953
initial condition: [0.9279438930218264, 0.9255469883339486, 0.5779241207215675, 0.49220856709256305]
last point: [0.9246513170512369, 0.9256592691239435, 0.5790371398136245, 0.4926229127674284]
LSE: [-0.3286652927269168, -0.32870753795884505, -11.092789544985965, -16.073860135211902]
_________________

 <- 
index: 13
I0: -1.7660606060606037
U0: 0.29211076173426953
initial condition: [0.9246513170512369, 0.9256592691239435, 0.5790371398136245, 0.4926229127674284]
last point: [0.9213672077307189, 0.9257715489879491, 0.5801530205291939, 0.49303702144973777]
LSE: [-0.3308527702818458, -0.33165119755037725, -11.064252000285023, -1

LSE: [-0.308755140833937, -0.30970703429422825, -11.252432988175226, -15.802980349780716]
_________________

 <- 
index: 6
I0: -1.7224242424242413
U0: 0.28946277590722036
initial condition: [0.9528367886141551, 0.9248307424479159, 0.5686803567455055, 0.4895599897009993]
last point: [0.9494830486528183, 0.9249427956029348, 0.5697692878713152, 0.4899749625567099]
LSE: [-0.3116485774334566, -0.3110335502176289, -11.223916414412413, -15.855139948846375]
_________________

 <- 
index: 7
I0: -1.727272727272726
U0: 0.28946277590722036
initial condition: [0.9494830486528183, 0.9249427956029348, 0.5697692878713152, 0.4899749625567099]
last point: [0.9461390209496345, 0.9250547188333693, 0.5708613722577696, 0.4903892274435622]
LSE: [-0.3132330147018198, -0.31378823732111194, -11.195671761869297, -15.907068552715893]
_________________

 <- 
index: 8
I0: -1.7321212121212106
U0: 0.28946277590722036
initial condition: [0.9461390209496345, 0.9250547188333693, 0.5708613722577696, 0.4903892274435622]
l

U0: 0.2868404151709937
initial condition: [20.569429832715436, 0.38935096953873527, 0.7617930814153301, 0.06578246881771595]
last point: [22.031941902263984, 0.40484428341080697, 0.7545014181007196, 0.06529173283832873]
LSE: [-0.00024392839196012665, -0.5555555937491304, -0.8603152194998596, -7.573190943009957]
----------------------

<<<<<<<<<<<<<<<<<<<<<
 <- 
index: 1
I0: -1.6884848484848483
U0: 0.2868404151709937
initial condition: [22.031941902263984, 0.40484428341080697, 0.7545014181007196, 0.06529173283832873]
last point: [8.67373284528595, 0.3957830307977219, 0.7784085163739942, 0.06960181552653302]
LSE: [-0.00012112777172222303, -0.5555556631867223, -0.6637067437934715, -7.5837732692900115]
_________________

 <- 
index: 2
I0: -1.693333333333333
U0: 0.2868404151709937
initial condition: [8.67373284528595, 0.3957830307977219, 0.7784085163739942, 0.06960181552653302]
last point: [2.501632143782908, 0.8409202225382579, 0.6482311705294448, 0.08478761349079812]
LSE: [-0.000440348988

LSE: [-0.3104799760043876, -0.31128808714633094, -11.10839883683992, -16.105401984997467]
_________________

 <- 
index: 9
I0: -1.7224242424242406
U0: 0.28553884421193876
initial condition: [0.9502334795432203, 0.9251015348688869, 0.5680177792164584, 0.49056244162940327]
last point: [0.9468991404249212, 0.9252122488788037, 0.5691072510054302, 0.4909719125440935]
LSE: [-0.3128816497716568, -0.3133452045247393, -11.082903931569623, -16.1538245580657]
_________________

 <- 
index: 10
I0: -1.7272727272727253
U0: 0.28553884421193876
initial condition: [0.9468991404249212, 0.9252122488788037, 0.5691072510054302, 0.4909719125440935]
last point: [0.9435738941377516, 0.9253228905399388, 0.5701997435532731, 0.4913808922891543]
LSE: [-0.3157566852606754, -0.3150464493320159, -11.057534048123005, -16.202195842842453]
_________________

 <- 
index: 11
I0: -1.73212121212121
U0: 0.28553884421193876
initial condition: [0.9435738941377516, 0.9253228905399388, 0.5701997435532731, 0.4913808922891543]
la

U0: 0.282954921111946
initial condition: [19.592082162702045, 0.40713621468433125, 0.7466769114633227, 0.07943413122626296]
last point: [0.6765735824973325, 0.9582774672536492, 0.4217219197954479, 0.4187011575451817]
LSE: [0.001545558662271712, -0.1284076838234539, -3.4408731275406104, -15.201980480967947]
_________________

 <- 
index: 4
I0: -1.6884848484848478
U0: 0.282954921111946
initial condition: [0.6765735824973325, 0.9582774672536492, 0.4217219197954479, 0.4187011575451817]
last point: [2.0799962547263173, 0.8474245117763561, 0.6490315107066249, 0.26912575934123206]
LSE: [0.00017309767139443564, -0.17806096043910888, -4.53220617376999, -18.318351379252146]
_________________

 <- 
index: 5
I0: -1.6933333333333325
U0: 0.282954921111946
initial condition: [2.0799962547263173, 0.8474245117763561, 0.6490315107066249, 0.26912575934123206]
last point: [0.965286822701784, 0.924727454937181, 0.5621797677638924, 0.48917727865395527]
LSE: [-0.3012009096759855, -0.3017355644463574, -11.143

In [7]:
function move_to_left( I0, U0, initial_condition_function, step )
    
    println("");flush(stdout)
    println("<<<<<<<<<<<<<<<<<<<<<");flush(stdout)
    index_left = 1
    
    while line(I0) < U0 && I0 > -1.8
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
            I0 = I0 - step
        end
        
        println(" <- ")
        print_output(index_left, I0, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 2, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
        separate()
        I0 = I0 - step
        index_left += 1
    end
end

move_to_left (generic function with 1 method)